In [91]:
%load_ext autoreload
%autoreload 2

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
model_name = 'gpt2'

# prompt management

In [93]:
dataset = (
    ('Rome', 'Italy'),
    ('Madrid', 'Spain'),
    ('Amsterdam', 'Netherlands'),
    ('Moscow', 'Russia'),
    ('Nairobi', 'Kenya'),
    ('Paris', 'France'),
    ('Tehran', 'Iran'),
    ('Tokyo', 'Japan'),
    ('Warsaw', 'Poland'),
    ('Ottawa', 'Canada'),
    ('Oslo', 'Norway'),
    ('Lisbon', 'Portugal'),
    ('Helsinki', 'Finland'),
    ('Havana', 'Cuba'),
    ('Doha', 'Qatar'),
    ('Damascus', 'Syria'),
    ('Canberra', 'Australia'),
    ('Cairo', 'Egypt'),
    ('Bern', 'Switzerland'),
    ('Berlin', 'Germany'),
    ('Beijing', 'China'),
    ('Athens', 'Greece'),
    ('Ankara', 'Turkey'),
)

# models

In [147]:
import sys
sys.path.append('..')
torch.set_grad_enabled(False)

from src.utils.model_utils import load_gpt_model_and_tokenizer, set_seed, get_submodule
from src.extraction import split_activation, extract_activations, get_mean_activations
from src.utils.prompt_helper import build_prompt_txt, tokenize_from_template, tokenize_ICL
set_seed(32)

In [95]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model, config = load_gpt_model_and_tokenizer(model_name)
print(model)
print(config)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2AttentionAltered(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
          (query): WrapperModule()
          (key): WrapperModule()
          (value): WrapperModule()
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
{'

Calculate the average of the activation on the dataset to grasp any pattern that is task-dependant and not token-dependant 

Activations have the following shape (num_layers, num_heads, seq_len, d_head), being each OV circuit output.

Still to answer:
- How this is connected to the residual stream?

Trying to use [nnsight](https://github.com/JadenFiotto-Kaufman/nnsight) and explore to know how it works

In [90]:
# select number of ICL examples
ICL_examples = 2

tok_ret, ids_ret = tokenize_ICL(tokenizer, ICL_examples = ICL_examples, dataset = dataset)
print(tok_ret[0])
print(ids_ret[0])

tensor([50256,    48,    25,    49,   462,   198,    32,    25, 45001,   628,
           48,    25, 18454,  6058,   198,    32,    25, 45355,   628,    48,
           25,  5840, 22506,   198,    32,    25])
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25]


___


In [152]:
mean_activation, outputs = get_mean_activations(
    tokenized_prompts=tok_ret,
    important_ids=ids_ret,
    model=model,
    config=config,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
